In [1]:
import pandas as pd
import io
from google.cloud import storage
# Create a client
client = storage.Client()

# Define your bucket name
bucket_name = "dsgt-clef-pan-2024"
# Get the bucket
bucket = client.get_bucket(bucket_name)

# List objects in the bucket
blobs = bucket.list_blobs()
for blob in blobs:
    print(blob.name)

JackGraphData/graph_test.tar.gz
JackGraphData/graph_train.tar.gz
JackGraphData/graph_val.tar.gz
PAN Data/Artem.pkl
PAN Data/GPT-wiki-intro.csv
PAN Data/GPT.pkl
PAN Data/HFD.pkl
PAN Data/HF_NabeelShar_ai_and_human_text.csv
PAN Data/HF_artem9k_ai_text_detection_pile/train-00000-of-00007-bc5952582e004d67.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00001-of-00007-71c80017bc45f30d.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00002-of-00007-ee2d43f396e78fbc.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00003-of-00007-529931154b42b51d.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00004-of-00007-b269dc49374a2c0b.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00005-of-00007-3dce5e05ddbad789.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00006-of-00007-3d8a471ba0cf1c8d.parquet
PAN Data/HF_dmitva_human_ai_generated_text.csv
PAN Data/JackGraphData/
PAN Data/Nabeel.pkl
PAN Data/PAN Competition.zip
PAN Data/competition_dataset/datase

In [2]:
from datasets import DatasetDict
file_path = "gs://dsgt-clef-pan-2024/PAN Data/competition_dataset/"
#content = blob.download_as_string()

trainTestValid_dataset = DatasetDict.load_from_disk(file_path)

/home/abindal/pan-2024/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(trainTestValid_dataset)

DatasetDict({
    train: Dataset({
        features: ['text_pair', 'label'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['text_pair', 'label'],
        num_rows: 15000
    })
    valid: Dataset({
        features: ['text_pair', 'label'],
        num_rows: 15000
    })
})


# Preprocess

In [57]:
from transformers import RobertaTokenizerFast
import pyarrow as pa
import torch
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

def tokenize_function(examples, tokenizer=tokenizer, max_length=250):
    tokenized_example = []
    for example, label in zip(examples['text_pair'], examples['label']):
        tokens1 = tokenizer.encode_plus(
            example['text1'],
            max_length=max_length,  
            padding='max_length', 
            truncation=True,
            return_tensors='pt',
            return_attention_mask=True
        )
        
        tokens2 = tokenizer.encode_plus(
            example['text2'],
            max_length=max_length,  
            padding='max_length', 
            truncation=True,
            return_tensors='pt',
            return_attention_mask=True
        )

        sep_token_id = tokenizer.sep_token_id

        input_ids = torch.cat((tokens1['input_ids'], torch.tensor([[sep_token_id]]), tokens2['input_ids']), dim=1)
        attention_mask = torch.cat((tokens1['attention_mask'], torch.ones(tokens1['attention_mask'].shape[0], 1), tokens2['attention_mask']), dim=1)

        #print(input_ids.size())
        #print(attention_mask.size())
        #tokens['label']=label
        tokenized_example.append({
            'input_ids': input_ids.transpose(0,1).cpu().numpy().tolist(),
            'attention_mask': attention_mask.transpose(0,1).cpu().numpy().tolist(),
            'label': label
        })
    
    df = pd.DataFrame(tokenized_example)
    tokenized_example = pa.Table.from_pandas(df)
    return tokenized_example
    
tokenized_datasets = trainTestValid_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 15000/15000 [01:28<00:00, 169.96 examples/s]


In [58]:
print(len(tokenized_datasets['train'][0]['attention_mask']))
print(len(tokenized_datasets['train'][0]['input_ids']))

501
501


In [36]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 15000
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 15000
    })
})


In [59]:
%pip install evaluate
import numpy as np
import evaluate
from peft import LoraConfig, TaskType, get_peft_model
# Get roberta model for classification from Hugging Face
from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [62]:
import optuna

#def training_function(trial: optuna.Trial):
def training_function():
    lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS, 
        r=1, #OG:1 
        lora_alpha=1,
        lora_dropout=0.1
        #lora_alpha=trial.suggest_float("lora_alpha", low = 0.1, high = 0.1), #OG:1, h = 10 
        #lora_dropout=trial.suggest_float("lora_dropout", low=0.1, high=0.1), #OG:0.1, h = 0.4
    )
    model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
    model = get_peft_model(model, lora_config)

    training_args = TrainingArguments(
        output_dir="train/LoRA-tune/", 
        evaluation_strategy="epoch",
        num_train_epochs=10,
        #learning_rate=trial.suggest_float("learning_rate",1e-4, 1e-2), #OG:1e-3, 1e-4, 1e-2
        learning_rate=1e-3,
        per_device_train_batch_size=24,
        per_device_eval_batch_size=24,
        weight_decay=0.01,
        #weight_decay=trial.suggest_float("weight_decay", 0.01, 0.01), #OG:0.01, 0.001, 0.1
        save_strategy="epoch",
        load_best_model_at_end=True,
        )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(20000)),
        eval_dataset = tokenized_datasets['valid'].shuffle(seed=42).select(range(4000)),
        compute_metrics=compute_metrics,
    )

    result = trainer.train()
    return result.training_loss

# Hyperparameter Tuning

In [61]:
# We want to minimize the loss! 
study = optuna.create_study(study_name='hyper-parameter-search', direction='minimize') 

# Optimize the objective using 5 different trials 
study.optimize(func=training_function, n_trials=1)   

# Gives the best loss value 
print(study.best_value) 

# Gives the best hyperparameter values to get the best loss value 
print(study.best_params) 

# Return info about best Trial such as start and end datetime, hyperparameters  
print(study.best_trial)

[I 2024-03-28 06:51:34,591] A new study created in memory with name: hyper-parameter-search
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[W 2024-03-28 06:51:35,684] Trial 0 failed with parameters: {'lora_alpha': 0.1, 'lora_dropout': 0.1, 'learning_rate': 0.008069197310233815, 'weight_decay': 0.01} because of the following error: ValueError('too many values to unpack (expected 2)').
Traceback (most recent call last):
  File "/home/abindal/pan-2024/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_223123/3950807290.py", line 33, in training_function
    result = trainer.train()
  File 

ValueError: too many values to unpack (expected 2)

In [ ]:
model.save_pretrained("train/LoRA/")

In [ ]:
# Run inference on the test dataset
predictions = trainer.predict(tokenized_datasets['test'].shuffle(seed=42))
print(predictions)
#Check the accuracy of predictions
print(compute_metrics(predictions))

PredictionOutput(predictions=array([[-0.05063864,  0.04590682],
       [-0.05064901,  0.04584191],
       [-0.05066186,  0.04588119],
       ...,
       [-0.05066562,  0.04585219],
       [-0.05068301,  0.04590267],
       [-0.05063813,  0.04588834]], dtype=float32), label_ids=array([0, 1, 1, ..., 1, 0, 1]), metrics={'test_loss': 0.6931726932525635, 'test_accuracy': 0.5118, 'test_runtime': 285.2511, 'test_samples_per_second': 52.585, 'test_steps_per_second': 2.191})


ValueError: too many values to unpack (expected 2)

In [ ]:
#Print test accuracy
print(predictions)

PredictionOutput(predictions=array([[-0.05063864,  0.04590682],
       [-0.05064901,  0.04584191],
       [-0.05066186,  0.04588119],
       ...,
       [-0.05066562,  0.04585219],
       [-0.05068301,  0.04590267],
       [-0.05063813,  0.04588834]], dtype=float32), label_ids=array([0, 1, 1, ..., 1, 0, 1]), metrics={'test_loss': 0.6931726932525635, 'test_accuracy': 0.5118, 'test_runtime': 285.2511, 'test_samples_per_second': 52.585, 'test_steps_per_second': 2.191})


In [ ]:
print(predictions.metrics)

{'test_loss': 0.6931726932525635, 'test_accuracy': 0.5118, 'test_runtime': 285.2511, 'test_samples_per_second': 52.585, 'test_steps_per_second': 2.191}
